In [1]:
import sys
sys.path.append("../")
# import time
# time.sleep(7200)

In [2]:
config = {
    "pretrained_model": "bert-base-cased",
    "tokenizer": "bert-base-cased",
    "max_seq_length": 512,
    "batch_size": 8,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 2,
    "is_multilabel": False,
    "valid_metric": "f1",
    "model_save_dir": "../checkpoints/bert_cased_512_bio_26k_20200908/",
    "patience": 4,
}

In [3]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [5]:
import pandas as pd
data_df = pd.read_csv("../datasets/biocls_26k.csv")
data_df.docs = data_df.docs.apply(lambda x: x.replace("\n",". ").replace("\r",""))
data_df.labels = data_df.labels.apply(lambda x: "01" if x==0 else "10")
data_df.head()

,Unnamed: 0,index,docs,labels,source,flag
0,0,32,India's Deccan gets more planes. . . . Air...,01,bbc,train
1,1,10431,. . DECLINES COMMENT ON HONGKONG LAND REPOR...,01,Reuters,train
2,2,1676,Letter to XPO staff reveals extra safety measu...,10,promed_extended,train
3,3,13747,. . . . KEYCORP <KEY> 3RD QTR NET. . . ...,01,Reuters,train
4,4,4435,. . . . API SAYS DISTILLATE STOCKS OFF 2.5...,01,Reuters,train


In [2]:
len(data_df.index)

25956

In [6]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [7]:
train_set.to_csv("../covid19-classification/hedwig-data/datasets/Promed_26k/train.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")
dev_set.to_csv("../covid19-classification/hedwig-data/datasets/Promed_26k/dev.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")
test_set.to_csv("../covid19-classification/hedwig-data/datasets/Promed_26k/test.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")

In [6]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(20764, 2596, 2596)

In [7]:
cls.train(train_set,dev_set)

Evaluating:   0%|          | 1/325 [00:00<00:40,  8.09it/s]

Execute [train_an_epoch] method costing 949608.61 ms
[Epoch 0] loss: 488.13574178144336



Evaluating: 100%|██████████| 325/325 [00:40<00:00,  8.04it/s]


Execute [eval] method costing 40443.88 ms
------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------+----------+
|    | metrics   |   values |
|----+-----------+----------|
|  0 | accuracy  | 0.98151  |
|  1 | precision | 0.962751 |
|  2 | recall    | 0.90566  |
|  3 | f1        | 0.933333 |
+----+-----------+----------+
------------------------------------------------------------


Evaluating:   0%|          | 1/325 [00:00<00:40,  8.04it/s]

...

Execute [eval] method costing 39740.55 ms



Evaluating:   0%|          | 1/325 [00:00<00:40,  8.09it/s]

Execute [train_an_epoch] method costing 939980.49 ms
[Epoch 9] loss: 36.3948739990592



Epoch:  90%|█████████ | 9/10 [2:44:12<18:14, 1094.67s/it]

Execute [eval] method costing 39729.71 ms
Early Stopping. Epoch: 9, best_valid_metric (f1): 0.9678284182305631


In [8]:
result = cls.test(test_set)
result

Evaluating: 100%|██████████| 325/325 [00:38<00:00,  8.33it/s]

Execute [eval] method costing 38999.15 ms


{'accuracy': 0.9892141756548536,
 'precision': 0.9405940594059405,
 'recall': 0.9895833333333334,
 'f1': 0.964467005076142}

In [10]:
import csv
with open('../results/bert_cased_512_bio_26k_20200908_test.csv', 'w') as f:
    for key in result.keys():
        f.write("%s,%s\n"%(key,result[key]))